<a href="https://colab.research.google.com/github/ichrakbouafif/Gotham/blob/main/NameBasedRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
recipes = pd.read_csv("/content/recipes.csv")

In [31]:
recipes.head()

,id_recette,nom,description,type,nb_personne,tmps_preparation,tmps_cuisson,difficulte,calories,nutrition,instruction,ing,img_instruction,propriete,ingredients,cout_total,cout_par_personne,image,rating,nb_vote
0,0,Mini croissants apéritif (pizza au jambon et f...,Des invités arrivent à l'improviste ? Pas de s...,Apéritif,8 parts,15 min,20 min,facile,192 Kcal,"{'Glucides': '12.3g', 'Lipides': '13.3g', 'Gra...",['Déroulez votre pâte feuilletée et coupez là ...,"['1 pâte feuilletée', 'Sauce tomate', 'Jambon'...",['https://www.ptitchef.com/imgupl/recipe-step/...,"['Sans sucre ajouté', 'Sans fruit à coque']","{'No header found': ['1 pâte feuilletée', 'Sau...",2.56€,(0.32€/part),['https://www.ptitchef.com/imgupl/recipe/mini-...,4.4,264.0
1,1,Roulés feuilletés apéritifs saumon basilic,"Pour un apéritif rapide, facile et savoureux, ...",Apéritif,8 parts,20 min,20 min,facile,225 Kcal,"{'Glucides': '11.3g', 'Lipides': '17.1g', 'Gra...","[""Si votre pâte feuilletée est ronde, étalez-l...","['1 pâte feuilletée', '150g de tranches de sau...",['https://www.ptitchef.com/imgupl/recipe-step/...,"['Sans sucre ajouté', 'Sans fruit à coque']","{'No header found': ['1 pâte feuilletée', '150...",9.31€,(1.16€/part),['https://www.ptitchef.com/imgupl/recipe/roule...,4.4,139.0
2,2,Roulés de saumon au chèvre et asperges,Des amuse-bouches sans cuisson hyper faciles à...,Apéritif,8 parts,20 min,--:--,facile,110 Kcal,"{'Glucides': '0.7g', 'Lipides': '7.4g', 'Gras ...",['Disposer un film plastique sur votre plan de...,"['6 tranches de saumon fumé', '200 gr de froma...",['https://www.ptitchef.com/imgupl/recipe-step/...,"['Sans gluten', 'Sans sucre ajouté', 'Sans oeuf']",{'No header found': ['6 tranches de saumon fum...,--,--,['https://www.ptitchef.com/imgupl/recipe/roule...,4.5,101.0
3,3,Torsades de pesto et parmesan,"La torsade feuilletée, c'est LA star de l'apér...",Apéritif,6 parts,10 min,10 min,facile,240 Kcal,"{'Glucides': '14.8g', 'Lipides': '18.1g', 'Gra...","['Étalez le pesto sur votre pâte feuilletée, p...","['1 pâte feuilletée', '2c. à soupe de pesto (2...",['https://www.ptitchef.com/imgupl/recipe-step/...,"['Végétarien', 'Sans sucre ajouté']","{'No header found': ['1 pâte feuilletée', '2c....",1.8€,(0.3€/part),['https://www.ptitchef.com/imgupl/recipe/torsa...,4.5,97.0
4,4,Petits choux au saumon,Une recette trop chou pour mettre en scène vos...,Apéritif,8 parts,30 min,25 min,moyen,156 Kcal,"{'Glucides': '8.6g', 'Lipides': '9g', 'Gras sa...","[""Dans une casserole, mettre l'eau à chauffer ...","['80 gr de farine', '50 gr de beurre', ""13 cl ...",['https://www.ptitchef.com/imgupl/recipe-step/...,"['Sans sucre ajouté', 'Sans fruit à coque']","{'La pâte à choux ': ['80 gr de farine', '50 g...",9.78€,(1.22€/part),['https://www.ptitchef.com/imgupl/recipe/petit...,4.4,93.0


In [37]:
def get_similar_recipes(recipe_name, top_n=5):
    # Convert input recipe name to lowercase
    recipe_name = recipe_name.lower()

    # Convert recipe names in DataFrame to lowercase
    recipes['nom_lower'] = recipes['nom'].str.lower()

    recipes['recipe_features'] = recipes['nom'] + ' ' + recipes['type'] + ' ' + recipes['difficulte'] + ' ' + \
                                  recipes['ing'] + ' ' + recipes['propriete']

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(recipes['recipe_features'])

    # Compute cosine similarity between recipes
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Check if the given recipe name exists in the DataFrame
    if recipe_name not in recipes['nom_lower'].values:
        print("Recipe '{}' not found.".format(recipe_name))
        return

    # Get the index of the target recipe
    recipe_index = recipes.index[recipes['nom_lower'] == recipe_name].tolist()[0]

    # Get the similarity scores for the target recipe
    similarity_scores = list(enumerate(cosine_sim[recipe_index]))

    # Sort the recipes based on similarity scores
    similar_recipes = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Extract top N similar recipes (excluding the recipe itself)
    similar_recipes = similar_recipes[1:top_n + 1]

    # If there are no similar recipes found
    if not similar_recipes:
        print("No similar recipes found for '{}'.".format(recipe_name))
    else:
        # Print the top N similar recipes
        for idx, score in similar_recipes:
            print("Recipe:", recipes.iloc[idx]['nom'])
            print("Similarity Score:", score)
            print("Recipe Features:", recipes.iloc[idx]['recipe_features'])
            print()




In [42]:
get_similar_recipes("Croissants feuilletés jambon / fromage frais")

Recipe: Croissants feuilletés à la béchamel, au jambon et au fromage
Similarity Score: 0.5612329808475081
Recipe Features: Croissants feuilletés à la béchamel, au jambon et au fromage Apéritif facile ['1 pâte feuilletée', '1 tranche de jambon', '125 gr de fromage râpé', '16 c. à café de béchamel', "1 jaune d'oeuf"] ['Sans sucre ajouté', 'Sans fruit à coque']

Recipe: Palmiers au jambon et fromage frais aux herbes
Similarity Score: 0.5213440286623026
Recipe Features: Palmiers au jambon et fromage frais aux herbes Apéritif facile ['1 rouleau de pâte feuilletée', '5 tranches fines de jambon blanc', '1 fromage aux herbes de Provence et échalotes ( type Tartare ou Boursin)'] ['Sans sucre ajouté', 'Sans fruit à coque']

Recipe: Mini croissants apéritif (pizza au jambon et fromage)
Similarity Score: 0.5125049295544671
Recipe Features: Mini croissants apéritif (pizza au jambon et fromage) Apéritif facile ['1 pâte feuilletée', 'Sauce tomate', 'Jambon', 'Fromage râpé (gruyère, mozzarella, ...)',

In [39]:
get_similar_recipes("cake au chorizo")

Recipe: Cake jambon & gruyère
Similarity Score: 0.4992538336024342
Recipe Features: Cake jambon & gruyère Entrée facile ['180 gr de farine', '3 oeufs', '8 cl d’huile d’olive', '10 cl de lait', '1 sachet de levure', '150 gr de dés de jambon', '60 gr de gruyère (râpé)', 'Beurre', 'Sel', 'Poivre'] ['Sans sucre ajouté', 'Sans fruit à coque']

Recipe: Cake au thon et à la tomate
Similarity Score: 0.4989979798399782
Recipe Features: Cake au thon et à la tomate Apéritif facile ['1 boîte de thon', '5 tomates', '100 gr de farine', '3 oeufs', '10 cl de lait', "10 cl d'huile", '100 gr de gruyère râpé', '1 sachet de levure chimique', 'sel/poivre/herbes de provence'] ['Sans sucre ajouté', 'Sans fruit à coque']

Recipe: Petites bouchées au Chorizo pour apéritif sympa !
Similarity Score: 0.48372192055321933
Recipe Features: Petites bouchées au Chorizo pour apéritif sympa ! Apéritif facile ['100 g de farine', '1/2 cuillère à café de levure chimique', '2 oeufs', '8 cl de lait', "5 cl d'huile (d'arachid

TypeError: no supported conversion for types: (dtype('O'),)